In this notebook we will show that any connected sum with at most 9 crossings shares no 0-surgery with a knot of up to 19 crossings. For that we recall that the 0-surgery on a connected sum is a JSJ manifold. So we will first check all knots with the same Alexander polynomial for beeing non-hyperbolic.

In [1]:
import snappy
import regina
import csv
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def change_notation(dt_code):
    """
    Changes Dowker-Thistlewait notation from alphabetical to numerical
    Input:
        dt_code (string): alphabetical DT notation
    Return:
        (string): numerical DT notation
    """
    alpha = "abcdefghijklmnopqrstuvwxyz"
    Alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    result = []
    for letter in dt_code:
        if letter in alpha:
            result.append(2* (alpha.index(letter) + 1))
        elif letter in Alpha:
            result.append(-2 * (Alpha.index(letter) + 1))
        else:
            print(dt_code)
    return "DT: " + str([tuple(result)])

def fill_triangulation(M):
    '''
    Fills all cusps but one.
    '''
    if M.num_cusps()==1:
        return M
    M=M.filled_triangulation([0])
    M=fill_triangulation(M)
    return M

#### This is Dunfield's util.py from his exceptional census

####  for a snappy manifold M descibed as a single filling of a cusp (so do filled_triangulation() as needed) 
####  the command regina_name(M) gives what regina identifies M as

"""

This file provides functions for working with Regina (with a little
help from SnapPy) to:

1. Give a standard name ("identify") manifolds, especially Seifert and
   graph manifolds.

2. Find essential tori.

3. Try to compute the JSJ decomposition.

"""

import regina
import snappy
import re
import networkx as nx

def appears_hyperbolic(M):
    acceptable = ['all tetrahedra positively oriented',
                  'contains negatively oriented tetrahedra']
    return M.solution_type() in acceptable and M.volume() > 0

def children(packet):
    child = packet.firstChild()
    while child:
        yield child
        child = child.nextSibling()

def to_regina(data):
    if hasattr(data, '_to_string'):
        data = data._to_string()
    if isinstance(data, str):
        if data.find('(') > -1:
            data = closed_isosigs(data)[0]
        return regina.Triangulation3(data)
    assert isinstance(data, regina.Triangulation3)
    return data

def extract_vector(surface):
    """
    Extract the raw vector of the (almost) normal surface in Regina's
    NS_STANDARD coordinate system.
    """
    S = surface
    T = S.triangulation()
    n = T.countTetrahedra()
    ans = []
    for i in range(n):
        for j in range(4):
            ans.append(S.triangles(i, j))
        for j in range(3):
            ans.append(S.quads(i, j))
    A = regina.NormalSurface(T, regina.NS_STANDARD, ans)
    assert A.sameSurface(S)
    return ans

def haken_sum(S1, S2):
    T = S1.triangulation()
    assert S1.locallyCompatible(S2)
    v1, v2 = extract_vector(S1), extract_vector(S2)
    sum_vec = [x1 + x2 for x1, x2 in zip(v1, v2)]
    A = regina.NormalSurface(T, regina.NS_STANDARD, sum_vec)
    assert S1.locallyCompatible(A) and S2.locallyCompatible(A)
    assert S1.eulerChar() + S2.eulerChar() == A.eulerChar()
    return A


def census_lookup(regina_tri):
    """
    Should the input triangulation be in Regina's census, return the
    name of the manifold, dropping the triangulation number.
    """
    hits = regina.Census.lookup(regina_tri)
    hit = hits.first()
    if hit is not None:
        name = hit.name()
        match = re.search('(.*) : #\d+$', name)
        if match:
            return match.group(1)
        else:
            return match

def standard_lookup(regina_tri):
    match = regina.StandardTriangulation.isStandardTriangulation(regina_tri)
    if match:
        return match.manifold()

def closed_isosigs(snappy_manifold, trys=20, max_tets=50):
    """
    Generate a slew of 1-vertex triangulations of a closed manifold
    using SnapPy.
    
    >>> M = snappy.Manifold('m004(1,2)')
    >>> len(closed_isosigs(M, trys=5)) > 0
    True
    """
    M = snappy.Manifold(snappy_manifold)
    assert M.cusp_info('complete?') == [False]
    surgery_descriptions = [M.copy()]

    try:
        for curve in M.dual_curves():
            N = M.drill(curve)
            N.dehn_fill((1,0), 1)
            surgery_descriptions.append(N.filled_triangulation([0]))
    except snappy.SnapPeaFatalError:
        pass

    if len(surgery_descriptions) == 1:
        # Try again, but unfill the cusp first to try to find more
        # dual curves.
        try:
            filling = M.cusp_info(0).filling
            N = M.copy()
            N.dehn_fill((0, 0), 0)
            N.randomize()
            for curve in N.dual_curves():
                D = N.drill(curve)
                D.dehn_fill([filling, (1,0)])
                surgery_descriptions.append(D.filled_triangulation([0]))
        except snappy.SnapPeaFatalError:
            pass

    ans = set()
    for N in surgery_descriptions:
        for i in range(trys):
            T = N.filled_triangulation()
            if T._num_fake_cusps() == 1:
                n = T.num_tetrahedra()
                if n <= max_tets:
                    ans.add((n, T.triangulation_isosig(decorated=False)))
            N.randomize()

    return [iso for n, iso in sorted(ans)]

def best_match(matches):
    """
    Prioritize the most concise description that Regina provides to
    try to avoid things like the Seifert fibered space of a node being
    a solid torus or having several nodes that can be condensed into a
    single Seifert fibered piece.
    """
    
    def score(m):
        if isinstance(m, regina.SFSpace):
            s = 0
        elif isinstance(m, regina.GraphLoop):
            s = 1
        elif isinstance(m, regina.GraphPair):
            s = 2
        elif isinstance(m, regina.GraphTriple):
            s = 3
        elif m is None:
            s = 10000
        else:
            s = 4
        return (s, str(m))
    return min(matches, key=score)

def identify_with_torus_boundary(regina_tri):
    """
    Use the combined power of Regina and SnapPy to try to give a name
    to the input manifold.
    """
    
    kind, name = None, None
    
    P = regina_tri.clone()
    P.finiteToIdeal()
    P.intelligentSimplify()
    M = snappy.Manifold(P.isoSig())
    M.simplify()
    if appears_hyperbolic(M):
        for i in range(100):
            if M.solution_type() == 'all tetrahedra positively oriented':
                break
            M.randomize()
        
        if not M.verify_hyperbolicity(bits_prec=100):
            raise RuntimeError('Cannot prove hyperbolicity for ' +
                               M.triangulation_isosig())
        kind = 'hyperbolic'
        ids = M.identify()
        if ids:
            name = ids[0].name()
    else:
        match = standard_lookup(regina_tri)
        if match is None:
            Q = P.clone()
            Q.idealToFinite()
            Q.intelligentSimplify()
            match = standard_lookup(Q)
        if match is not None:
            kind = match.__class__.__name__
            name = str(match)
        else:
            name = P.isoSig()
    return kind, name
            
    
    

def is_toroidal(regina_tri):
    """
    Checks for essential tori and returns the pieces of the
    associated partial JSJ decomposition.
    
    >>> T = to_regina('hLALAkbccfefgglpkusufk')  # m004(4,1)
    >>> is_toroidal(T)[0]
    True
    >>> T = to_regina('hvLAQkcdfegfggjwajpmpw')  # m004(0,1)
    >>> is_toroidal(T)[0]
    True
    >>> T = to_regina('nLLLLMLPQkcdgfihjlmmlkmlhshnrvaqtpsfnf')  # 5_2(10,1)
    >>> T.isHaken()
    True
    >>> is_toroidal(T)[0]
    False

    Note: currently checks all fundamental normal tori; possibly
    the theory lets one just check *vertex* normal tori.
    """
    T = regina_tri
    assert T.isZeroEfficient()
    surfaces = regina.NNormalSurfaceList.enumerate(T,
                          regina.NS_QUAD, regina.NS_FUNDAMENTAL)
    for i in range(surfaces.size()):
        S = surfaces.surface(i)
        if S.eulerChar() == 0:
            if not S.isOrientable():
                S = S.doubleSurface()
            assert S.isOrientable()
            X = S.cutAlong()
            X.intelligentSimplify()
            X.splitIntoComponents()
            pieces = list(children(X))
            if all(not C.hasCompressingDisc() for C in pieces):
                ids = [identify_with_torus_boundary(C) for C in pieces]
                return (True, sorted(ids))
                
    return (False, None)


def decompose_along_tori(regina_tri):
    """
    First, finds all essential normal tori in the manifold associated
    with fundamental normal surfaces.  Then takes a maximal disjoint
    collection of these tori, namely the one with the fewest tori
    involved, and cuts the manifold open along it.  It tries to
    identify the pieces, removing any (torus x I) components. 

    Returns: (has essential torus, list of pieces)

    Note: This may fail to be the true JSJ decomposition because there
    could be (torus x I)'s in the list of pieces and it might well be
    possible to amalgamate some of the pieces into a single SFS.
    """
    
    T = regina_tri
    assert T.isZeroEfficient()
    essential_tori = []
    surfaces = regina.NNormalSurfaceList.enumerate(T,
                          regina.NS_QUAD, regina.NS_FUNDAMENTAL)
    for i in range(surfaces.size()):
        S = surfaces.surface(i)
        if S.eulerChar() == 0:
            if not S.isOrientable():
                S = S.doubleSurface()
            assert S.isOrientable()
            X = S.cutAlong()
            X.intelligentSimplify()
            X.splitIntoComponents()
            pieces = list(children(X))
            if all(not C.hasCompressingDisc() for C in pieces):
                essential_tori.append(S)

    if len(essential_tori) == 0:
        return False, None
    
    D = nx.Graph()
    for a, A in enumerate(essential_tori):
        for b, B in enumerate(essential_tori):
            if a < b:
                if A.disjoint(B):
                    D.add_edge(a, b)

    cliques = list(nx.find_cliques(D))
    if len(cliques) == 0:
        clique = [0]
    else:
        clique = min(cliques, key=len)
    clique = [essential_tori[c] for c in clique]
    A = clique[0]
    for B in clique[1:]:
        A = haken_sum(A, B)

    X = A.cutAlong()
    X.intelligentSimplify()
    X.splitIntoComponents()
    ids = [identify_with_torus_boundary(C) for C in list(children(X))]
    # Remove products
    ids = [i for i in ids if i[1] not in ('SFS [A: (1,1)]', 'A x S1')]
    return (True, sorted(ids))

def regina_name(closed_snappy_manifold, trys=100):
    """
    >>> regina_name('m004(1,0)')
    'S3'
    >>> regina_name('s006(-2, 1)')
    'SFS [A: (5,1)] / [ 0,-1 | -1,0 ]'
    >>> regina_name('m010(-1, 1)')
    'L(3,1) # RP3'
    >>> regina_name('m022(-1,1)')
    'SFS [S2: (3,2) (3,2) (4,-3)]'
    >>> regina_name('v0004(0, 1)')
    'SFS [S2: (2,1) (4,1) (15,-13)]'
    >>> regina_name('m305(1, 0)')
    'L(3,1) # RP3'
    """
    M = snappy.Manifold(closed_snappy_manifold)
    isosigs = closed_isosigs(M, trys=trys, max_tets=25)
    if len(isosigs) == 0:
        return
    T = to_regina(isosigs[0])
    if T.isIrreducible():
        if T.countTetrahedra() <= 11:
            for i in range(3):
                T.simplifyExhaustive(i)
                name = census_lookup(T)
                if name is not None:
                    return name
            
        matches = [standard_lookup(to_regina(iso)) for iso in isosigs]
        match = best_match(matches)
        if match is not None:
            return str(match)
    else:
        T.connectedSumDecomposition()
        pieces = [regina_name(P) for P in children(T)]
        if None not in pieces:
            return ' # '.join(sorted(pieces))
        
def recognize_mfd(knot):
    """
    Uses regina and snappy to recognize the name of its 0-filling.
    """
    K=snappy.Manifold(knot)
    K_reg=regina_name(K)
    if K_reg is not None:
        return K_reg  
    else:
        try:
            K_reg=decompose_along_tori(to_regina(closed_isosigs(K)[0]))
        except TypeError:
            K_reg=None
        if K_reg is not None and K_reg[0]==True:
            return 'JSJ'+str(K_reg[1])

<>:181: DeprecationWarning: invalid escape sequence \d
<>:181: DeprecationWarning: invalid escape sequence \d
<ipython-input-1-cef6550c8024>:181: DeprecationWarning: invalid escape sequence \d
  match = re.search('(.*) : #\d+$', name)


In [2]:
connected_sums=['3_1_plus_3_1','3_1_plus_4_1','3_1_plus_5_1','3_1_plus_5_2','3_1_plus_6_1','3_1_plus_6_2',
                '3_1_plus_6_3','4_1_plus_4_1','4_1_plus_5_1','4_1_plus_5_2','3_1_plus_3_1_plus_3_1']

possible_same_0_surgeries=[] 
for name in connected_sums:
    listname=[]
    try:
        with open(name+'.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                listname.append(row)
            listname=listname[1:]
            low_cros=[]
            high_cros=[]
            for x in listname:
                if int(x[1])<16:
                    low_cros.append(x)
                else:
                    high_cros.append(x)
            listname=[]
            listname.append(low_cros)
            listname.append(high_cros)
            possible_same_0_surgeries.append(listname)
    except FileNotFoundError:
        pass

In [4]:
len(possible_same_0_surgeries)

11

In [6]:
start_time = time.time()
possible_same_0_surgeries_both_prop_non_hyp=[]

for x in possible_same_0_surgeries:
    unclear=[]
    for [knot,cros,DT] in x[1]:
        K=snappy.Link(change_notation(DT)).exterior()
        K.dehn_fill((0,1))
        vol=better_volume(K)
        if vol==0:
            unclear.append([knot,cros,DT])
    possible_same_0_surgeries_both_prop_non_hyp.append([x[0],unclear]) 
    print('Possible equal surgeries:',[x[0],unclear])
print('Time taken: %s hours ' % ((time.time() - start_time)/3600))

Possible equal surgeries: [[['3_1_plus_3_1', '6', '']], [['16nh_0000011', '16', 'gNMLKJpIOFEDCBHa'], ['18ns_59', '18', 'hMeKgOcrJQDnBpFlIa'], ['18nh_00000014', '18', 'hPONMLKrJQGFEDCBIa'], ['18nh_00000312', '18', 'eFjGlKIqDnoAPcbRhM'], ['18nh_00000313', '18', 'cFkIgBoLmPaNqReJDh'], ['19nh_000000425', '19', 'kEhpSnIcMQaOGrJdLfB'], ['19nh_000000424', '19', 'jfpSnhLbOrMGQcIeKaD'], ['19nh_000001540', '19', 'ePlQsIOFnArdbhmGCkj']]]
Possible equal surgeries: [[['3_1_plus_4_1', '7', '']], []]
Possible equal surgeries: [[['3_1_plus_5_1', '8', '']], []]
Possible equal surgeries: [[['3_1_plus_5_2', '8', '']], []]
Possible equal surgeries: [[['3_1_plus_6_1', '9', '']], []]
Possible equal surgeries: [[['3_1_plus_6_2', '9', '']], []]
Possible equal surgeries: [[['3_1_plus_6_3', '9', '']], []]
Possible equal surgeries: [[['4_1_plus_4_1', '8', '']], []]
Possible equal surgeries: [[['4_1_plus_5_1', '9', '']], []]
Possible equal surgeries: [[['4_1_plus_5_2', '9', '']], []]
Possible equal surgeries: [[[

We work a bit harder with the remaining examples.

In [7]:
start_time = time.time()
still_unclear=[]
for x in possible_same_0_surgeries_both_prop_non_hyp:
    print(x[0])
    unclear=[]
    for [knot,cros,DT] in x[1]:
        K=snappy.Link(change_notation(DT)).exterior()
        K.dehn_fill((0,1))
        vol=better_volume(K,try_hard=True)
        print(knot,vol)
        if vol==0:
            unclear.append([knot,cros,DT])
    if len(unclear)>0:
        still_unclear.append([x[0],unclear])
    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

[['3_1_plus_3_1', '6', '']]
16nh_0000011 0
18ns_59 0
18nh_00000014 0
18nh_00000312 0
18nh_00000313 0
19nh_000000425 0
19nh_000000424 0
19nh_000001540 11.0645862261?
----------------
[['3_1_plus_4_1', '7', '']]
----------------
[['3_1_plus_5_1', '8', '']]
----------------
[['3_1_plus_5_2', '8', '']]
----------------
[['3_1_plus_6_1', '9', '']]
----------------
[['3_1_plus_6_2', '9', '']]
----------------
[['3_1_plus_6_3', '9', '']]
----------------
[['4_1_plus_4_1', '8', '']]
----------------
[['4_1_plus_5_1', '9', '']]
----------------
[['4_1_plus_5_2', '9', '']]
----------------
[['3_1_plus_3_1_plus_3_1', '9', '']]
----------------
Time taken: 0.8123347361882528 minutes 


The remaining 0-fillings we expect to be non-hyperbolic. We try to verify that by using the regina code.

In [8]:
still_unclear

[[[['3_1_plus_3_1', '6', '']],
  [['16nh_0000011', '16', 'gNMLKJpIOFEDCBHa'],
   ['18ns_59', '18', 'hMeKgOcrJQDnBpFlIa'],
   ['18nh_00000014', '18', 'hPONMLKrJQGFEDCBIa'],
   ['18nh_00000312', '18', 'eFjGlKIqDnoAPcbRhM'],
   ['18nh_00000313', '18', 'cFkIgBoLmPaNqReJDh'],
   ['19nh_000000425', '19', 'kEhpSnIcMQaOGrJdLfB'],
   ['19nh_000000424', '19', 'jfpSnhLbOrMGQcIeKaD']]]]

In [9]:
start_time = time.time()
for x in still_unclear:
    print(x[0])
    for [knot,cros,DT] in x[1]:
        K=snappy.Link(change_notation(DT)).exterior()
        K.dehn_fill((0,1))
        rec=recognize_mfd(K)
        print(knot,recognize_mfd(K))
    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

[['3_1_plus_3_1', '6', '']]
16nh_0000011 SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -4,5 | -3,4 ]
18ns_59 SFS [D: (2,1) (3,1)] U/m Non-or, g=1 + 2 punctures/n2 x~ S1 U/n SFS [D: (3,1) (3,2)], m = [ 1,-1 | 0,1 ], n = [ 2,1 | 1,1 ]
18nh_00000014 SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -5,6 | -4,5 ]
18nh_00000312 JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 's783')]
18nh_00000313 JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 's783')]
19nh_000000425 JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 's783')]
19nh_000000424 JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 's783')]
----------------
Time taken: 3.361183567841848 minutes 


So all these manifolds are really JSJ manifolds. But there pieces are not knot complemenets and thus they cannot be homeomorphic to the 0-surgery of a connected sum. This proves the result.